# 🚀 PipeZone - Getting Started

This notebook demonstrates how to use PipeZone to extract, transform, and load data.

## What you'll learn:
1. How to work with connections
2. How to execute flows
3. How to access data in MinIO
4. How to use Spark for transformations

## Setup

In [ ]:
import sys
sys.path.insert(0, '/home/pipezone/utils')

from connection_manager import ConnectionManager
from flow_executor import FlowExecutor
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import os
print(f"Working directory: {os.getcwd()}")
print(f"Python version: {sys.version}")

## 1️⃣ Working with Connections

In [ ]:
# Initialize connection manager
cm = ConnectionManager()

# List all available connections
connections = cm.list_connections()
print("📋 Available connections:")
for conn in connections:
    print(f"  • {conn}")

In [ ]:
# Load a specific connection configuration
config = cm.load_connection_config('minio_raw')
print(f"\n📦 Connection: {config['name']}")
print(f"   Type: {config['type']}")
print(f"   Description: {config['description']}")
print(f"   Bucket: {config['connection']['bucket']}")

In [ ]:
# Test connections
print("\n🔍 Testing connections:")
for conn in connections:
    try:
        status = "✅" if cm.test_connection(conn) else "❌"
        print(f"  {status} {conn}")
    except Exception as e:
        print(f"  ⚠️  {conn} - {str(e)}")

## 2️⃣ Working with Flows

In [ ]:
# List available flows
import os
from pathlib import Path

flows_path = '/opt/pipezone/metadata/flows'
flows = [f.stem for f in Path(flows_path).glob('*.yml')]

print("📊 Available flows:")
for flow in flows:
    print(f"  • {flow}")

In [ ]:
# Load a flow configuration
executor = FlowExecutor()

if flows:
    flow_config = executor.load_flow_config(flows[0])
    print(f"\n📋 Flow: {flow_config['name']}")
    print(f"   Description: {flow_config['description']}")
    print(f"   Source: {flow_config['source']['connection']}")
    print(f"   Target: {flow_config['target']['connection']}")
    print(f"   Schedule: {flow_config.get('schedule', {}).get('cron', 'Not scheduled')}")

## 3️⃣ Using Spark

In [ ]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("PipeZone Demo") \
    .master(os.getenv('SPARK_MASTER', 'local[*]')) \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(f"✅ Spark version: {spark.version}")
print(f"   Master: {spark.sparkContext.master}")

In [ ]:
# Create sample data
from datetime import datetime, timedelta
import random

# Generate sample customer data
sample_data = []
for i in range(1000):
    sample_data.append({
        'customer_id': i + 1,
        'name': f'Customer {i + 1}',
        'email': f'customer{i + 1}@example.com',
        'country': random.choice(['UZ', 'KZ', 'TJ', 'KG', 'TM']),
        'created_at': datetime.now() - timedelta(days=random.randint(0, 365)),
        'updated_at': datetime.now() - timedelta(hours=random.randint(0, 24))
    })

df = spark.createDataFrame(sample_data)
print(f"\n📊 Created sample dataset with {df.count()} records")
df.show(5)

In [ ]:
# Analyze data
print("\n📈 Data Analysis:")
df.groupBy('country').count().orderBy('count', ascending=False).show()

print("\n📊 Schema:")
df.printSchema()

In [ ]:
# Apply transformations
df_transformed = df \
    .withColumn('email_domain', F.split(F.col('email'), '@').getItem(1)) \
    .withColumn('year', F.year('created_at')) \
    .withColumn('month', F.month('created_at')) \
    .withColumn('processed_at', F.current_timestamp())

print("\n✨ Transformed data:")
df_transformed.select('customer_id', 'email', 'email_domain', 'country', 'year', 'month').show(5)

## 4️⃣ Working with MinIO (S3)

In [ ]:
# Write data to MinIO (S3)
minio_path = "s3a://raw/demo/customers/"

df_transformed.write \
    .format('parquet') \
    .mode('overwrite') \
    .partitionBy('country', 'year', 'month') \
    .option('compression', 'snappy') \
    .save(minio_path)

print(f"\n✅ Data written to MinIO: {minio_path}")

In [ ]:
# Read data back from MinIO
df_read = spark.read \
    .format('parquet') \
    .load(minio_path)

print(f"\n📖 Read {df_read.count()} records from MinIO")
df_read.show(5)

In [ ]:
# Query data from MinIO
print("\n🔍 Query: Customers from Uzbekistan:")
df_read.filter(F.col('country') == 'UZ') \
    .select('customer_id', 'name', 'email', 'created_at') \
    .show(10)

## 🎯 Next Steps

1. **Create your own connections** in `metadata/connections/`
2. **Define data flows** in `metadata/flows/`
3. **Test flows** using `FlowExecutor`
4. **Schedule flows** in Airflow by setting `schedule.enabled: true`
5. **Monitor executions** in MySQL `pipeline_execution_logs` table

## 📚 Resources

- MinIO Console: http://localhost:9001
- Airflow UI: http://localhost:8080
- Spark UI: http://localhost:8081
- Documentation: See README.md

Happy data engineering! 🚀